In [133]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [134]:
df = spark.read.options(delimiter=";", inferSchema='True', header=True).csv("/home/mario/Downloads/10minutenwerte_only_100/produkt_zehn_min_tu_19900101_19991231_03366.txt")
df.printSchema()

root
 |-- STATIONS_ID: double (nullable = true)
 |-- MESS_DATUM: long (nullable = true)
 |-- QN: double (nullable = true)
 |-- PP_10: double (nullable = true)
 |-- TT_10: double (nullable = true)
 |-- TM5_10: double (nullable = true)
 |-- RF_10: double (nullable = true)
 |-- TD_10: double (nullable = true)



In [135]:
df.show(n=5)

+-----------+------------+---+-----+-----+------+-----+-----+
|STATIONS_ID|  MESS_DATUM| QN|PP_10|TT_10|TM5_10|RF_10|TD_10|
+-----------+------------+---+-----+-----+------+-----+-----+
|     3366.0|199001010000|1.0|972.6| -4.4|  -4.0| 94.0| -4.6|
|     3366.0|199001010010|1.0|972.5| -4.3|  -3.9| 94.0| -4.7|
|     3366.0|199001010020|1.0|972.4| -4.4|  -4.0| 94.0| -4.7|
|     3366.0|199001010030|1.0|972.3| -4.4|  -4.1| 94.0| -4.6|
|     3366.0|199001010040|1.0|972.3| -4.5|  -4.1| 94.0| -4.8|
+-----------+------------+---+-----+-----+------+-----+-----+
only showing top 5 rows



In [136]:
#df.withColumn("MESS_DATUM",df.MESS_DATUM.cast(StringType())) \
#  .show(truncate=False)

In [137]:
from datetime import datetime
from pyspark.sql.functions import col, udf, to_timestamp, year, month
from pyspark.sql.types import DoubleType, IntegerType, DateType, StringType
df = df.select("STATIONS_ID","MESS_DATUM","TT_10")
df = df.withColumn("MESS_DATUM",col("MESS_DATUM").cast(StringType()))
df = df.withColumn("STATIONS_ID",col("STATIONS_ID").cast(IntegerType()))
df = df.withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))

In [138]:
df.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- TT_10: double (nullable = true)



In [139]:
df.show(n=5)

+-----------+-------------------+-----+
|STATIONS_ID|         MESS_DATUM|TT_10|
+-----------+-------------------+-----+
|       3366|1990-01-01 00:00:00| -4.4|
|       3366|1990-01-01 00:10:00| -4.3|
|       3366|1990-01-01 00:20:00| -4.4|
|       3366|1990-01-01 00:30:00| -4.4|
|       3366|1990-01-01 00:40:00| -4.5|
+-----------+-------------------+-----+
only showing top 5 rows



Bauernregel:

„Wenn's im Dezember nicht wintert, sommert's im Juni auch nicht.“

Hierbei handelt es sich im Prinzip um die entgegengesetzte Regel zu der vorherigen:
Im Juni wird es nicht warm, wenn im Dezember kein richtiger Winter war.
Es stimmt, dass die These zu 65 Prozent eintrifft. Andersrum, wenn es also im Juni nicht „sommert“, wird es im Dezember nicht „wintern“,
stimmt die Bauernregel aber nicht. Warum das so ist, weiß keiner so genau.


In [140]:
df.describe().show()


+-------+-----------+------------------+
|summary|STATIONS_ID|             TT_10|
+-------+-----------+------------------+
|  count|     480758|            480758|
|   mean|     3366.0|-37.06368068758087|
| stddev|        0.0|209.83299914047205|
|    min|       3366|            -999.0|
|    max|       3366|              35.6|
+-------+-----------+------------------+



In [141]:
# we only need months June and Decemeber
df = df.filter((month("MESS_DATUM") == 6) | (month("MESS_DATUM") == 12))
df.show()

+-----------+-------------------+-----+
|STATIONS_ID|         MESS_DATUM|TT_10|
+-----------+-------------------+-----+
|       3366|1990-06-01 00:00:00|  6.7|
|       3366|1990-06-01 00:10:00|  6.9|
|       3366|1990-06-01 00:20:00|  6.2|
|       3366|1990-06-01 00:30:00|  5.8|
|       3366|1990-06-01 00:40:00|  5.8|
|       3366|1990-06-01 00:50:00|  6.0|
|       3366|1990-06-01 01:00:00|  6.1|
|       3366|1990-06-01 01:10:00|  6.0|
|       3366|1990-06-01 01:20:00|  5.9|
|       3366|1990-06-01 01:30:00|  5.6|
|       3366|1990-06-01 01:40:00|  5.7|
|       3366|1990-06-01 01:50:00|  6.3|
|       3366|1990-06-01 02:00:00|  6.5|
|       3366|1990-06-01 02:10:00|  6.8|
|       3366|1990-06-01 02:20:00|  5.8|
|       3366|1990-06-01 02:30:00|  5.3|
|       3366|1990-06-01 02:40:00|  5.5|
|       3366|1990-06-01 02:50:00|  5.5|
|       3366|1990-06-01 03:00:00|  6.5|
|       3366|1990-06-01 03:10:00|  7.3|
+-----------+-------------------+-----+
only showing top 20 rows



In [142]:
df = df.select("STATIONS_ID", year("MESS_DATUM").alias("year"),month("MESS_DATUM").alias("month"), "TT_10")
df.show()

+-----------+----+-----+-----+
|STATIONS_ID|year|month|TT_10|
+-----------+----+-----+-----+
|       3366|1990|    6|  6.7|
|       3366|1990|    6|  6.9|
|       3366|1990|    6|  6.2|
|       3366|1990|    6|  5.8|
|       3366|1990|    6|  5.8|
|       3366|1990|    6|  6.0|
|       3366|1990|    6|  6.1|
|       3366|1990|    6|  6.0|
|       3366|1990|    6|  5.9|
|       3366|1990|    6|  5.6|
|       3366|1990|    6|  5.7|
|       3366|1990|    6|  6.3|
|       3366|1990|    6|  6.5|
|       3366|1990|    6|  6.8|
|       3366|1990|    6|  5.8|
|       3366|1990|    6|  5.3|
|       3366|1990|    6|  5.5|
|       3366|1990|    6|  5.5|
|       3366|1990|    6|  6.5|
|       3366|1990|    6|  7.3|
+-----------+----+-----+-----+
only showing top 20 rows



In [143]:
# calculate average temperature for each year+month
df = df.groupBy("year", "month").avg("TT_10")
df.show()

+----+-----+--------------------+
|year|month|          avg(TT_10)|
+----+-----+--------------------+
|1990|    6|  13.595092592592565|
|1990|   12| -11.698342803030272|
|1991|    6|  14.654583333333356|
|1992|    6|  -66.83006666666667|
|1993|    6|  14.659828465461315|
|1994|    6|   16.76222737819026|
|1993|   12|  2.4830121021963185|
|1995|   12| -1.2314964157706108|
|1994|   12|    2.02283535217586|
|1995|    6|  14.494441871236706|
|1996|    6|  17.114129273504336|
|1996|   12|  -4.925990752972258|
|1997|   12|  0.8322356630824369|
|1998|    6|   17.09474912485418|
|1997|    6|   16.27305555555556|
|1999|    6|  15.699606481481515|
|1998|   12| -1.1544822949350064|
|1999|   12|-0.18109441578829447|
+----+-----+--------------------+



In [144]:
df2 = df.filter(df.month == 12)
df2.show()

+----+-----+--------------------+
|year|month|          avg(TT_10)|
+----+-----+--------------------+
|1990|   12| -11.698342803030272|
|1993|   12|  2.4830121021963185|
|1995|   12| -1.2314964157706108|
|1994|   12|    2.02283535217586|
|1996|   12|  -4.925990752972258|
|1997|   12|  0.8322356630824369|
|1998|   12| -1.1544822949350064|
|1999|   12|-0.18109441578829447|
+----+-----+--------------------+



In [145]:
df2 = df2.rdd.map(lambda x: 
    df.filter("year" == x["year"]+1 & "month" == 6).select("temperature").alias("temperature_june_next_year")
    )
df2 = df2.toDF()
df2.show()

Traceback (most recent call last):
  File "/home/mario/.local/lib/python3.10/site-packages/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/mario/.local/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/mario/.local/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/home/mario/.local/lib/python3.10/site-packages/pyspark/context.py", line 447, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [81]:
df.createOrReplaceTempView("df")
sqlDF = spark.sql("SELECT * FROM df;")
sqlDF.show()

+------------+
|unique_years|
+------------+
|        1990|
|        1991|
|        1992|
|        1993|
|        1994|
|        1995|
|        1997|
|        1996|
|        1998|
|        1999|
+------------+



In [74]:
df = df.select(year('MESS_DATUM').alias("unique_years")).distinct()

In [75]:
df.show()

+------------+
|unique_years|
+------------+
|        1990|
|        1991|
|        1992|
|        1993|
|        1994|
|        1995|
|        1997|
|        1996|
|        1998|
|        1999|
+------------+



In [78]:
unique_years_list = df.select('unique_years').rdd.flatMap(lambda x: x).collect()
print(unique_years_list)

[1990, 1991, 1992, 1993, 1994, 1995, 1997, 1996, 1998, 1999]


In [5]:
df.write.option("header",True).csv("/home/mario/Downloads/10minutenwerte/all_together_first_half")

22/12/15 09:34:54 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19900101_19991231_03366.txt


22/12/15 09:34:55 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19910101_19991231_03023.txt


22/12/15 09:34:58 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19910101_19991231_01468.txt


22/12/15 09:34:58 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19910619_19991231_00691.txt


22/12/15 09:34:58 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19911031_19991231_00591.txt


...

22/12/15 09:42:50 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19981222_19991231_05871.txt
22/12/15 09:42:50 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19950828_19991231_01975.txt
22/12/15 09:42:53 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19960531_19991231_05516.txt
22/12/15 09:42:53 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minuten

22/12/15 09:43:08 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19971119_19991231_00701.txt
22/12/15 09:43:12 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19970101_19991231_05404.txt
22/12/15 09:43:13 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19950101_19991231_00433.txt
22/12/15 09:43:14 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minuten

22/12/15 09:43:19 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19950929_19991231_04931.txt
22/12/15 09:43:19 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19960401_19991231_04928.txt
22/12/15 09:43:20 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19960828_19991231_02812.txt


22/12/15 09:43:20 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19970205_19991231_02928.txt
22/12/15 09:43:23 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19971209_19991231_04094.txt
22/12/15 09:43:23 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19950531_19991231_00282.txt
22/12/15 09:43:25 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minuten

22/12/15 09:43:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19950501_19991231_05371.txt
22/12/15 09:43:35 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19951201_19991231_03032.txt
22/12/15 09:43:36 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minutenwerte_only_half/produkt_zehn_min_tu_19980225_19991231_01832.txt
22/12/15 09:43:37 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 9
CSV file: file:///home/mario/Downloads/10minuten